In [1]:
#CREDITOS: 
#OLAYA MIGUEL
#CRUZ PEDRO
#GARCÍA AGUSTÍN 
#FECHA DE DESARROLLO: 20 DE AGOSTO DE 2023
#PROYECTO DE SERVICIO
#UNAM
#REPOSITORIO GITHUB: https://github.com/AntonioOlay/Interfaz-Gr-fica-de-Visualizaci-n-Rapida-usando-NCAR-Vapor-en-JupyterLab

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
    
from vapor import session, renderer, dataset, camera, transferfunction, utils
from vapor.animation import Animation
import os
from numpy import cross, eye, dot, radians, asarray, array
from matplotlib import pyplot as plt
import cv2
from ipywidgets import Button

from numpy import cross, eye, dot, radians, asarray, array
from scipy.linalg import expm, norm

Vapor 3.8.3
Python 3.9.17 (/home/a.7023/mambaforge/envs/vapor3.8.3)
OpenGL 4.6.0 NVIDIA 470.129.06


In [2]:
files = os.listdir() #No se especifica directorio, ya que al estar la casilla vacia, por defecto usa el directorio actual
ncfiles=[]
for i in files:
    if os.path.isfile(os.path.join(i)) and i.endswith('.nc'):
        ncfiles.append(i)
        
nc=widgets.Dropdown(
    options=ncfiles,
    layout={'width': 'max-content'}, # If the items' names are long
    value=ncfiles[0],
    style={'description_width': 'initial'},
    description='Archivo NC: ',
)

        
class IsoRenderer:
    def __init__(self, nc):
        self.nc = nc
        self.ses = session.Session() #Creamos una sesion para almacenar todos los objetos
        self.data = self.ses.OpenDataset(dataset.WRF, [nc])
        
        self.mapa = self.data.NewRenderer(renderer.ImageRenderer)
        
        self.barb = self.data.NewRenderer(renderer.BarbRenderer)
        self.barb.SetDimensions(2)
        self.barvars = ["U10","V10"] #VARIABLES DEL RENDERIZADO "BARBS"
        self.colobarvar="T2" #VARIABLE QUE SE USA PARA COLOREAR EL RENDERIZADO BARBS
        
        self.flow = self.data.NewRenderer(renderer.FlowRenderer)
        self.flow.SetDimensions(3)
        self.flowvars = ["U","V","W"] #VARIABLES DEL RENDERIZADO "FLOW"
        self.colorflowvar="T" #VARIABLE QUE SE USA PARA COLOREAR EL RENDERIZADO FLOW
        
        
        self.ren1 = self.data.NewRenderer(renderer.VolumeIsoRenderer)

        self.ren2 = self.data.NewRenderer(renderer.VolumeIsoRenderer)
        
        
    def vars3d(self) -> iter:
        self.vars3d = self.data.GetDataVarNames(3)
        return self.vars3d

    def V(self, tiff,isovar1,isovar2):
        
        self.isovar1=isovar1
        self.isovar2=isovar2
        
        ##############################################
        
        self.ren1.SetVariableName(self.isovar1)   
        
        ##############################################
        
        self.ren2.SetVariableName(self.isovar2)   
        
        ##############################################
        
        self.barb.SetFieldVariableNames(self.barvars)
        self.barb.SetLineThickness(0.7) 
        self.barb.SetLengthScale(0.8)
        self.barb.SetColorMapVariableName(self.colobarvar)
        self.barb.SetXBarbsCount(28)
        self.barb.SetYBarbsCount(28)        
        
        ##############################################
        
        self.flow.SetFieldVariableNames(self.flowvars)
        self.flow.SetGridNumOfSeeds([5,5,3])
        self.flowrake = self.flow.GetRakeRegion() #Creamos un objeto para modificar la region donde se plantaran las semillas
        RER = self.flowrake.GetExtents() #Regresa una lista de los limites de la region actual del rake
        self.flowrake.SetExtents((RER[0][0], RER[0][1], RER[0][2]), (RER[1][0], RER[1][1], (RER[1][2])/2)) #Indicamos los limites de la region RAKE SET REGION RAKE
        self.flow.SetRenderGeom3D(1)
        self.flow.SetRenderRadiusScalar(1.5)

        
        ##############################################
        
        
        
        
        ##############################################
        
        
        
        
        ##############################################
        
        
        
        
        ##############################################

    #sacar variable
    def vars2d(self) -> iter:
        self.vars2d = self.data.GetDataVarNames(2) 
        return self.vars2d
        
    def T(self, timestep):
        self.ts = self.data.GetNumTimeSteps()
        self.timestep=timestep
        return self.ts
        
    def C(self,colorft1, isovalue1, colorft2, isovalue2, colorftbarb, colorback, colorftflow):
        
        self.ft1 = self.ren1.GetPrimaryTransferFunction()
        self.ft2 = self.ren2.GetPrimaryTransferFunction()
        self.ftb = self.barb.GetPrimaryTransferFunction()
        self.ftf = self.flow.GetPrimaryTransferFunction()
        
        self.colorft1=colorft1
        self.isovalue1=isovalue1
        
        self.colorft2=colorft2
        self.isovalue2=isovalue2
        
        self.colorftbarb=colorftbarb
        
        self.colorftflow=colorftflow
        
        self.colorback=colorback
        if colorback == (1,1,1):
            self.colortime = (0,0,0) 
        if colorback == (0,0,0):
            self.colortime = (1,1,1) 
        
    def maxiso1(self) -> float:
        self.maxiso1=self.ft1.GetMaxMapValue()
        return self.maxiso1
    
    def miniso1(self) -> float:
        self.miniso1=self.ft1.GetMinMapValue()
        return self.miniso1
    
    def maxiso2(self) -> float:
        self.maxiso2=self.ft2.GetMaxMapValue()
        return self.maxiso2
    
    def miniso2(self) -> float:
        self.miniso2=self.ft2.GetMinMapValue()
        return self.miniso2

        
    def O(self,y11, y21, y31, y41, y12, y22, y32, y42):
        self.y11=y11
        self.y21=y21
        self.y31=y31
        self.y41=y41
        
        self.y12=y12
        self.y22=y22
        self.y32=y32
        self.y42=y42
        
    def TR(self,Zoom,valhei,EnaM,Ena,Ena2,EnaB,EnaF):
        self.Zoom = Zoom
        self.valhei = valhei
        self.EnaM=EnaM
        self.Ena=Ena
        self.Ena2=Ena2
        self.EnaB=EnaB
        self.EnaF=EnaF
        
        
    def CAM(self,ANGCAM):
        self.cam = self.ses.GetCamera()
        self.cp=self.cam.GetPosition()
        self.cd=self.cam.GetDirection()
        self.up=self.cam.GetUp()
        if ANGCAM == 'VISTA DESDE ARRIBA':
            self.cam.ViewAll()
        if ANGCAM == 'INCLINADO':
            self.cp=(self.cp[0],self.cp[1]*0.85, self.cp[2]*0.75)
            self.cd=[self.cd[0],(self.cd[1]+1)*0.65, self.cd[2]*(-0.75)]
            self.up=(self.up[0],self.up[1]*0.75, (self.up[2]+1)*0.65)
        if ANGCAM == 'MUY INCLINADO':
            self.cp=(self.cp[0],self.cp[1]*0.80, self.cp[2]*0.55)
            self.cd=[self.cd[0],(self.cd[1]+1)*0.80, self.cd[2]*(-0.55)]
            self.up=(self.up[0],self.up[1]*0.55, (self.up[2]+1)*0.80)
        
        
    def SALIDA(self):
        self.EnaM
        self.Ena
        self.Ena2
        self.EnaB
        self.isovar1
        self.isovar2
        
        print("Antes de realizar la animacion asegurate de realizar todos los ajustes necesarios","\nUna vez se haya realizado el renderizado, para seguir con la modificación; seleccionar el MODO VISUALIZACION antes de actualizar")
        
        self.mapa.SetIsGeoRef(1)
        self.mapa.SetImagePath(tiff.value)
        
        self.ren1.SetUseColormapVariable(1)
        self.ren1.SetColorMapVariableName(self.isovar1)
        self.ft1.LoadBuiltinColormap(colorft1.value) 
        self.ren1.SetIsoValues([self.isovalue1]) 
        
        self.ren2.SetUseColormapVariable(1)
        self.ren2.SetColorMapVariableName(self.isovar2)
        self.ft2.LoadBuiltinColormap(colorft2.value) 
        self.ren2.SetIsoValues([self.isovalue2]) 
        
        self.barb.SetColorMapVariableName(self.colobarvar)
        self.ftb.LoadBuiltinColormap(colorftbarb.value) 
        
        self.flow.SetColorMapVariableName(self.colorflowvar)
        self.ftf.LoadBuiltinColormap(colorftflow.value)
        
        
        
  ######################################################################      
        
        
        self.opacities1 = [(self.miniso1, self.y11), (self.maxiso1*(1/3), self.y21), (self.maxiso1*(2/3), self.y31), (self.maxiso1, self.y41)] 
        self.ft1.SetOpacityControlPoints(self.opacities1) #Aplicamos estos vectores de opacidad al renderizado
        
        self.opacities2 = [(self.miniso2, self.y12), (self.maxiso2*(1/3), self.y22), (self.maxiso2*(2/3), self.y32), (self.maxiso2, self.y42)] 
        self.ft2.SetOpacityControlPoints(self.opacities2) #Aplicamos estos vectores de opacidad al renderizado
        

        
######################################################

        self.sc=self.mapa.GetTransform().GetScales() 
        self.sc[2] = self.valhei 
        self.mapa.GetTransform().SetScales(self.sc) 
        self.ren1.GetTransform().SetScales(self.sc)
        self.ren2.GetTransform().SetScales(self.sc)
        self.barb.GetTransform().SetScales(self.sc)
        self.flow.GetTransform().SetScales(self.sc)
        
    #################################################################    
        self.origen=self.mapa.GetTransform().GetOrigin()
        self.origen[2] = 0 
        self.mapa.GetTransform().SetOrigin(self.origen) 
        self.ren1.GetTransform().SetOrigin(self.origen)
        self.ren2.GetTransform().SetOrigin(self.origen)
        self.barb.GetTransform().SetOrigin(self.origen)
        self.flow.GetTransform().SetOrigin(self.origen)
        
        
##############################################################

        self.trans=self.barb.GetTransform().GetTranslations()
        self.trans[2]=5000
        self.barb.GetTransform().SetTranslations(self.trans)
        
##############################################################


        self.cam
        self.cam.ViewAll()
        self.cam.LookAt(self.cp,self.mapa.GetTransform().GetOrigin(),self.up)
        self.cam.Zoom(self.Zoom)

##############################################################
    
        notes = self.ses.GetSceneAnnotations() #Creamos un objeto con el modulo de anotaciones
        notes.SetUseDomainFrame(0) #Quitamos las lineas blancas que representan al espacio 3D
        notes.SetTimeType(notes.TimeAnnotationType.Formatted) #Agregamos a la escena y especificamos 
        notes.SetUseRegionFrame(1)
        notes.SetTimeColor(self.colortime) #Cambiamos el color de la representacion del tiempo a BLANCO
        notes.SetBackgroundColor(self.colorback)

                            
        cobar = self.ren1.GetColorbarAnnotation() #Guardamos el ColorBar en un objeto
        cobar.SetEnabled(1) #LO hacemos visible
        cobar.SetSize((0.1, 0.1)) #Definimos su tamaño
        cobar.SetTitle(self.isovar1) #Agregamos un titulo
        cobar.SetCornerPosition((0.95, 0.95))  #Indicamos la posicion en la escena donde se mostrara
        
        cobar2 = self.ren2.GetColorbarAnnotation() #Guardamos el ColorBar en un objeto
        cobar2.SetEnabled(1) #LO hacemos visible
        cobar2.SetSize((0.1, 0.1)) #Definimos su tamaño
        cobar2.SetTitle(self.isovar2) #Agregamos un titulo
        cobar2.SetCornerPosition((0.95, 0.05))  #Indicamos la posicion en la escena donde se mostrara
        
        #############################################
        
        self.mapa.SetEnabled(self.EnaM)
        self.ren1.SetEnabled(self.Ena)
        self.ren2.SetEnabled(self.Ena2)
        self.barb.SetEnabled(self.EnaB)
        self.flow.SetEnabled(self.EnaF)
        
        #################################################
                            
        self.mapa.SetHeightVariableName("HGT")
        self.barb.SetHeightVariableName("HGT")
        
        
        #################################################
        
        print(self.timestep)
        self.ses.SetTimestep(self.timestep)
        print("Valor Minimo y Maximo de la variable", self.isovar1 ,": ",self.ft1.GetMinMapValue(),",",self.ft1.GetMaxMapValue())
        print("Valor Minimo y Maximo de la variable", self.isovar2 ,": ",self.ft2.GetMinMapValue(),",",self.ft2.GetMaxMapValue())
        
        self.ses.SetResolution(*[1280, 720])
        self.ses.Show()

        
        
        
    def A(self,kf,koa):
        self.nc
        self.ses
        self.ts
        print(koa)
        anim = Animation(self.ses)
        
        if kf == "Imagen .PNG":
            koaOPTIONS = ["MODO VISUALIZACION", "PNG Estatica", "PNG Con Rotación"]
        if kf == "Video .MP4":
            koaOPTIONS = ["MODO VISUALIZACION","MP4 Estatica", "MP4 Con Rotación"]
        if kf=="MODO VISUALIZACION":
            koaOPTIONS=["Estas en el modo visualizacion"]
        
        if koa=="MP4 Estatica":
            dur=30
            FR = round((dur * 15) / self.ts) #Si deseamos un numero de imagenes por segundo debemos intercambiar "FR" por "fr" 
            #en la ecuacion "f" y en ses.SetTimestep(int(i/fr))
            f = FR * self.ts #Numero de imagenes que se guardaran para crear el video completo
            for i in range(0, f):  
                self.ses.SetTimestep(int(i/FR)) #Con este comando cada TIMESTEP tendra un numero determinado de cuadros
                #dependiendo de la duracion del video elegida (si es que se elige una duracion expecifica)
                #si se elige un numero de imagenes por TS especifico entonces la duracion variara. 
                #En ambos casos los cuadros por TS varian con relacion al numero total de cuadros "f"
                #Para que el video muestre todos los TS y no se detengan o terminen hasta que concluya el video
                anim.CaptureFrame() #Comando para capturar cada cuadro 
                print(f"Rendering Animation [{'='*round(i/5)}{' '*round((f/5)-i/5)}] {(100*i)/(f+1):.0f}%", end="\r")
                
            
            anim.ShowInteractive()
            anim.SaveMP4("RenIsoSurfaceESTATICO.mp4") #Guardamos todos los cuadros en un archivo con formato .MP4

        if koa=="MP4 Con Rotación":
            dur=30
            video_framerate = 5 #Imagenes por timestep capturadas
            #video_resolution = (1280, 720)
            #ses.SetResolution(*video_resolution)
            ##########SI DESEAS SELECCIONAR UNA DURACION MAS PRECISA, DEBES SUTITUIR "framR" en lugar de "video_framrate"#######
            ######SI DESEAS FRAMERATE UN ESPECIFICO DEBES SUSTITUIR "video_framrate" en lugar de "framR"##########
            #Ambas sustituciones deben hacerse tanto en n_frames, como en  ses.SetTimestep(int(i / video_framerate))
            framR = round((dur * 15) / self.ts) #
            UseValueFromSessionFile = None
            rotate_speed = 5  # deg/s 
            rotation_axis = [0,0,1]  # Z (up)
            rotation_center = UseValueFromSessionFile  # Can be overridden with [x,y,z] coordinates
            save_individual_frames = False
            n_frames = framR * self.ts
            du = n_frames / 15 #Siempre sera el framerate original del metodo SaveMP4 y .Show
            print("Duracion del video: ", dur, "segundos")
            pos, dir, up, tgt = [asarray(x) for x in [self.cam.GetPosition(), self.cam.GetDirection(), self.cam.GetUp(), self.cam.GetTarget()]]
            if rotation_center:
                tgt = asarray(rotation_center)
            def rotation_matrix(axis, theta):
                return expm(cross(eye(3), axis / norm(axis) * theta))
            for i in range(0, n_frames):
                print(f"Rendering... [{'#'*round(40*i/(n_frames-1))}{' '*round(40*(1-i/(n_frames-1)))}] {100*(i+1)/n_frames:.0f}%",end="\r" if i < n_frames-1 else "\n")
                self.ses.SetTimestep(int(i / framR))
                M = rotation_matrix(rotation_axis, radians(rotate_speed) * i / video_framerate)
                self.cam.SetPosition(dot(M, pos - tgt) + tgt)
                self.cam.SetDirection(dot(M, dir))
                self.cam.SetUp(dot(M, up))
                anim.CaptureFrame()
                if save_individual_frames:
                    self.ses.Render(f"{output}_{i:04}.png")
                    
            anim.ShowInteractive()
            anim.SaveMP4('RenIsoSurfaceGIRATORIO.mp4')
            
        if koa == 'PNG Estatica':
            for i in range(0, self.ts):
                self.ses.SetTimestep(int(i))
                self.ses.RenderToImage().save("IsoSurEst"+ self.nc + self.isovar + str(i) + ".png")
                print(f"Guardando Imagenes... [{'#'*round(40*i/(self.ts-1))}{' '*round(40*(1-i/(self.ts-1)))}] {100*(i+1)/self.ts:.0f}%",end="\r" if i < self.ts-1 else "\n")
                
        if koa=="PNG Con Rotación":
            dur=30
            video_framerate = 5 #Imagenes por timestep capturadas
            #video_resolution = (1280, 720)
            #ses.SetResolution(*video_resolution)
            ##########SI DESEAS SELECCIONAR UNA DURACION MAS PRECISA, DEBES SUTITUIR "framR" en lugar de "video_
            ######SI DESEAS FRAMERATE UN ESPECIFICO DEBES SUSTITUIR "video_framrate" en lugar de "framR"########
            #Ambas sustituciones deben hacerse tanto en n_frames, como en  ses.SetTimestep(int(i / video_framera
            framR = round((dur * 15) / self.ts) #
            UseValueFromSessionFile = None
            rotate_speed = 5  # deg/s 
            rotation_axis = [0,0,1]  # Z (up)
            rotation_center = UseValueFromSessionFile  # Can be overridden with [x,y,z] coordinates
            save_individual_frames = False
            n_frames = framR * self.ts
            du = n_frames / 15 #Siempre sera el framerate original del metodo SaveMP4 y .Show
            pos, dir, up, tgt = [asarray(x) for x in [self.cam.GetPosition(), self.cam.GetDirection(), self.cam.GetUp(), self.cam.GetTarget()]]
            if rotation_center:
                tgt = asarray(rotation_center)
            def rotation_matrix(axis, theta):
                return expm(cross(eye(3), axis / norm(axis) * theta))
            for i in range(0,n_frames):
                M = rotation_matrix(rotation_axis, radians(rotate_speed) * i / video_framerate)
                self.cam.SetPosition(dot(M, pos - tgt) + tgt)
                self.cam.SetDirection(dot(M, dir))
                self.cam.SetUp(dot(M, up))
                self.ses.RenderToImage().save("IsoSurRot"+ self.nc + self.isovar + str(i) + ".png")
                print(f"Guardando Imagenes... [{'#'*round(40*i/(n_frames-1))}{' '*round(40*(1-i/(n_frames-1)))}] {100*(i+1)/n_frames:.0f}%",end="\r" if i < n_frames-1 else "\n")
        else:
            print("Antes de realizar la animacion asegurate de realizar todos los ajustes necesarios")
        
        return koaOPTIONS


In [3]:
tiffiles = os.listdir() #No se especifica directorio, ya que al estar la casilla vacia, por defecto usa el directorio actual
tfiles=[]
for i in tiffiles:
    if os.path.isfile(os.path.join(i)) and i.endswith(('.tiff','.tms')):
        tfiles.append(i)
        
        
tiff=widgets.Dropdown(
    options=tfiles,
    layout={'width': 'max-content'}, # If the items' names are long
    value=tfiles[0],
    style={'description_width': 'initial'},
    description='Mapa georreferenciado: ',
)


################################################################################################################


ts=40
timestep = widgets.BoundedIntText(
    value=20,
    min=0,
    max=ts,
    step=1,
    description='TimeStep',
    disabled=False,
    style={'description_width': 'initial'},
)

varsd3=[0]
isovar1 = widgets.Dropdown(
    options=varsd3,
    layout={'width': 'max-content'}, # If the items' names are long
    value=varsd3[0],
    style={'description_width': 'initial'},
    description='Variable Iso 1',
)

isovar2 = widgets.Dropdown(
    options=varsd3,
    layout={'width': 'max-content'}, # If the items' names are long
    value=varsd3[0],
    style={'description_width': 'initial'},
    description='Variable Iso 2',
)

ARCHIVOS = widgets.GridBox([nc, tiff, timestep,isovar1,isovar2],layout=widgets.Layout(grid_template_columns="repeat(3, 400px)"))


##################################################################################################

EnaM=widgets.Checkbox(
    value=1,
    description='Mostrar Imagen',
    disabled=False,
    indent=False
)

Ena = widgets.Checkbox(
    value=1,
    description='Mostrar IsoSurface 1',
    disabled=False,
    indent=False
)

Ena2 = widgets.Checkbox(
    value=1,
    description='Mostrar IsoSurface 2',
    disabled=False,
    indent=False
)

EnaB = widgets.Checkbox(
    value=1,
    description='Mostrar Barbs',
    disabled=False,
    indent=False
)

EnaF = widgets.Checkbox(
    value=1,
    description='Mostrar Flow',
    disabled=False,
    indent=False
)


RENDERIZADOS = widgets.GridBox([EnaM,Ena,Ena2,EnaB,EnaF],layout=widgets.Layout(grid_template_columns="repeat(4, 300px)"))

##################################################################################################

colorftlist=["Diverging/CoolWarm",'Sequential/Rainbow','Sequential/dense','Sequential/algae','Sequential/KindlmannExtended','Sequential/ice',
 'Sequential/BlackBodyExtended', 'Sequential/BlackWhite', 'Sequential/Kindlmann', 'Sequential/rain', 'Sequential/haline', 'Sequential/BlackBody', 'Sequential/thermal','Sequential/amp', 'Sequential/deep', 'Sequential/tempo', 'Sequential/phase',
 'Sequential/matter', 'Sequential/solar', 'Sequential/speed', 'Sequential/turbid', 'Highlighting/oxy', 'Highlighting/topo', 'Diverging/balance', 'Diverging/delta',
 'Diverging/PurpleWhiteOrange', 'Diverging/GreenWhiteRed', 'Diverging/CoolWarmBent', 'Diverging/tarn', 'Diverging/GreenWhitePurple',
 'Diverging/diff', 'Diverging/BlueWhiteGold', 'Diverging/CoolWarmSmooth',
 'Diverging/CoolWarm', 'Diverging/curl']

colorft1=widgets.Dropdown(
    options=colorftlist,
    value='Sequential/deep',
    layout={'width': 'max-content'},
    description='Color IsoRenderer1',
    style={'description_width': 'initial'},
    disabled=False,
)

miniso1=0
maxiso1=1
isovalue1 = widgets.FloatSlider(
    min=miniso1,
    max=maxiso1,
    value=(maxiso1/2),
    step=((maxiso1-miniso1)/1000000000),
    description='IsoSurface1',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.18f',
)

colorft2=widgets.Dropdown(
    options=colorftlist,
    value="Sequential/phase",
    layout={'width': 'max-content'},
    description='Color IsoRenderer2',
    style={'description_width': 'initial'},
    disabled=False,
)

miniso2=0
maxiso2=1
isovalue2 = widgets.FloatSlider(
    value=(maxiso2/2),
    min=miniso2,
    max=maxiso2,
    step=((maxiso2-miniso2)/1000000000),
    description='IsoSurface2',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.18f',
)

colorftbarb=widgets.Dropdown(
    options=colorftlist,
    value="Sequential/thermal",
    layout={'width': 'max-content'},
    description='Color BarbsRender',
    style={'description_width': 'initial'},
    disabled=False,
)

colorback=widgets.Dropdown(
    options=[('Negro',(0,0,0)), ('Blanco',(1,1,1))],
    value=(1,1,1),
    layout={'width': 'max-content'},
    description='Color de Fondo',
    style={'description_width': 'initial'},
    disabled=False,
)

colorftflow=widgets.Dropdown(
    options=colorftlist,
    value='Sequential/haline',
    layout={'width': 'max-content'},
    description='Color FlowRenderer',
    style={'description_width': 'initial'},
    disabled=False,
)


COLOR = widgets.GridBox([colorft1,colorft2,colorftbarb,colorftflow,isovalue1, isovalue2,colorback],layout=widgets.Layout(grid_template_columns="repeat(3, 350px)"))

##################################################################################################

y11=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O1 IsoSurface1',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y21=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O2 IsoSurface1',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y31=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O3 IsoSurface1',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y41=widgets.FloatSlider(
    value=0.6,
    min=0,
    max=1,
    step=0.000001,
    description='O4 IsoSurface1',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y12=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O1 IsoSurface2',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y22=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O2 IsoSurface2',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y32=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O3 IsoSurface2',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)

y42=widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.000001,
    description='O4 IsoSurface2',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True,
    readout_format='.8f',
)


OPACIDAD = widgets.GridBox([y11, y21, y31, y41, y12, y22, y32, y42],layout=widgets.Layout(grid_template_columns="repeat(4, 320px)"))

##################################################################################################

Zoom=widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.0001,
    description='Zoom',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.4f',
)

valhei=widgets.FloatText(
    value=5,
    description='Escala de la altura',
    style={'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

ANGCAM=widgets.Dropdown(
    options=['VISTA DESDE ARRIBA', 'INCLINADO', 'MUY INCLINADO'],
    layout={'width': 'max-content'}, # If the items' names are long
    value='VISTA DESDE ARRIBA',
    style={'description_width': 'initial'},
    description='POSICION Y VISTA DE LA CAMARA',
)


TRANSFORMACIONCAMARA = widgets.GridBox([Zoom, valhei,ANGCAM],layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))


######################################################################################################


kf = widgets.Dropdown(
    options=["MODO VISUALIZACION","Imagen .PNG", "Video .MP4"],
    layout={'width': 'max-content'}, # If the items' names are long
    value="MODO VISUALIZACION",
    style={'description_width': 'initial'},
    description='Formato del archivo multimedia: ',
)

koaOPTIONS=[""]
koa=widgets.Select(
    options=koaOPTIONS,
    value=koaOPTIONS[0], # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Tipo de animacion: ',
    style={'description_width': 'initial'},
    disabled=False
)



ANIMACION = widgets.GridBox([kf,koa],layout=widgets.Layout(grid_template_columns="repeat(2, 400px)"))

###########################################################################################################


UI = widgets.Tab()
UI.children = [ARCHIVOS, RENDERIZADOS, COLOR, OPACIDAD, TRANSFORMACIONCAMARA, ANIMACION]
UI.titles = ('ARCHIVOS', 'RENDERIZADOS', 'COLOR', 'OPACIDAD', 'TRANSFORMACION/CAMARA', "ANIMACION")

Check=widgets.ToggleButton(
    value=False,
    description='Actualizar',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [4]:
print("PARA APRENDER A AJUSTAR LAS CARACTERISTICAS DEL RENDERIZADO IR A: https://github.com/AntonioOlay/Interfaz-Gr-fica-de-Visualizaci-n-Rapida-usando-NCAR-Vapor-en-JupyterLab")


def ALLP(Check): 
    #################################################
    VolumenIso=IsoRenderer(nc.value)
    vars3d = VolumenIso.vars3d()
    isovar1.options=vars3d
    isovar2.options=vars3d
    VolumenIso.V(tiff.value,isovar1.value,isovar2.value)
    

    ts=VolumenIso.T(timestep.value)
    timestep.max=ts
###########################################
    colorftlist=VolumenIso.C(colorft1.value, isovalue1.value,colorft2.value, isovalue2.value, colorftbarb.value,colorback.value,colorftflow.value)
    maxiso1=VolumenIso.maxiso1()
    miniso1=VolumenIso.miniso1()
    
    maxiso2=VolumenIso.maxiso2()
    miniso2=VolumenIso.miniso2()
    
    isovalue1.max=maxiso1
    isovalue1.min=miniso1
    
    isovalue2.max=maxiso2
    isovalue2.min=miniso2
##########################################

    VolumenIso.O(y11.value,y21.value,y31.value,y41.value,y12.value,y22.value,y32.value,y42.value)

    VolumenIso.TR(Zoom.value,valhei.value,EnaM.value,Ena.value,Ena2.value,EnaB.value,EnaF.value)
    
    VolumenIso.CAM(ANGCAM.value)
    
    #####################################################
    
    display(UI)

    VolumenIso.SALIDA()
    
    koaOPTIONS=VolumenIso.A(kf.value, koa.value)
    koa.options=koaOPTIONS
    
    

widgets.interact(ALLP, Check=Check)

PARA APRENDER A AJUSTAR LAS CARACTERISTICAS DEL RENDERIZADO IR A: https://github.com/AntonioOlay/Interfaz-Gr-fica-de-Visualizaci-n-Rapida-usando-NCAR-Vapor-en-JupyterLab


interactive(children=(ToggleButton(value=False, button_style='info', description='Actualizar', icon='check', t…

<function __main__.ALLP(Check)>